In [7]:
from scrapernhl.qmjhl.scrapers.schedule import getSchedule, getSeasons
from scrapernhl.qmjhl.scrapers.teams import getTeams
from scrapernhl.qmjhl.scrapers.games import getAPIEvents, scrape_game
from scrapernhl.qmjhl.scrapers.schedule import getSchedule

import pandas as pd
import numpy as np

from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor

pd.set_option('display.max_columns', None)

# Reload module to get latest changes
import importlib
import scrapernhl.qmjhl.scrapers.games
importlib.reload(scrapernhl.qmjhl.scrapers.games)
from scrapernhl.qmjhl.scrapers.games import scrape_game

In [4]:
df1 = getAPIEvents(31171)  # Example game ID
df1 = pd.json_normalize(df1)

df1.query("event in ['shot', 'goal']")


,event,goalie_in_id,goalie_out_id,period_id,time,team_code,s,team_id,goalie_in_info.player_id,goalie_in_info.jersey_number,goalie_in_info.team_id,goalie_in_info.team_code,goalie_in_info.first_name,goalie_in_info.last_name,id,period,time_formatted,home_player_id,visitor_player_id,home_win,location_id,x_location,y_location,win_team_id,player_home.player_id,player_home.jersey_number,player_home.team_id,player_home.team_code,player_home.first_name,player_home.last_name,player_visitor.player_id,player_visitor.jersey_number,player_visitor.team_id,player_visitor.team_code,player_visitor.first_name,player_visitor.last_name,player_id,goalie_id,home,shot_type,shot_type_description,shot_quality_description,quality,game_goal_id,player_team_id,goalie_team_id,assist1_player_id,assist2_player_id,goal_type_name,player.player_id,player.jersey_number,player.team_id,player.team_code,player.first_name,player.last_name,goalie.player_id,goalie.jersey_number,goalie.team_id,goalie.team_code,goalie.first_name,goalie.last_name,goal_type,goal_player_id,location_set,power_play,empty_net,penalty_shot,short_handed,insurance_goal,game_winning,game_tieing,scorer_goal_num,plus,minus,goal_scorer.player_id,goal_scorer.jersey_number,goal_scorer.team_id,goal_scorer.team_code,goal_scorer.first_name,goal_scorer.last_name,assist1_player.player_id,assist1_player.jersey_number,assist1_player.team_id,assist1_player.team_code,assist1_player.first_name,assist1_player.last_name,assist2_player.player_id,assist2_player.jersey_number,assist2_player.team_id,assist2_player.team_code,assist2_player.first_name,assist2_player.last_name,hit_type,hitter.player_id,hitter.jersey_number,hitter.team_id,hitter.team_code,hitter.first_name,hitter.last_name,player_served,offence,pp,time_off_formatted,bench,minutes,minutes_formatted,penalty_class_id,penalty_class,lang_penalty_description,player_penalized_info.player_id,player_penalized_info.jersey_number,player_penalized_info.team_id,player_penalized_info.team_code,player_penalized_info.first_name,player_penalized_info.last_name,player_served_info.player_id,player_served_info.jersey_number,player_served_info.team_id,player_served_info.team_code,player_served_info.first_name,player_served_info.last_name,goalie_out_info.player_id,goalie_out_info.jersey_number,goalie_out_info.team_id,goalie_out_info.team_code,goalie_out_info.first_name,goalie_out_info.last_name
3,shot,NaN,NaN,1,00:17,NaN,17,3,NaN,NaN,NaN,NaN,NaN,NaN,625412,NaN,0:17,NaN,NaN,NaN,NaN,530.0,173.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18790,19488,0,5,Default,Quality goal,5,240034,3,2,21282,19913,,18790,91,3,Cap,Jacob,Newcombe,19488,34,2,Bat,Joshua,Fleming,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,goal,NaN,NaN,1,0:17,NaN,17,3,NaN,NaN,NaN,NaN,NaN,NaN,240034,1st,0:17,NaN,NaN,NaN,NaN,530.0,173.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21282,19913,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,18790,0,0,0,0,0,0,0,0,1,"[{'player_id': '19913', 'jersey_number': '16',...","[{'player_id': '20177', 'jersey_number': '29',...",18790,91,3,Cap,Jacob,Newcombe,21282,27,3,Cap,Romain,Litalien,19913,16,3,Cap,Lucas,Romeo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,shot,NaN,NaN,1,01:49,NaN,109,3,NaN,NaN,NaN,NaN,NaN,NaN,625413,NaN,1:49,NaN,NaN,NaN,NaN,393.0,61.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19104,19488,0,5,Default,Non quality on net,2,,3,2,NaN,NaN,NaN,19104,17,3,Cap,Xavier,Daigle,19488,34,2,Bat,Joshua,Fleming,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [ ]:
# Test the improved scrape_game function
df = scrape_game(31171)

print(f"Total events: {len(df)}")
print(f"\nEvent counts:\n{df['event'].value_counts()}")

# Show shots and goals (NHL-style: one row per shot event)
df.query("event in ['shot', 'goal', 'penaltyshot']").head(15)

,event,time,s,id,period,time_formatted,location_id,x_location,y_location,goalie_id,home,shot_type,shot_type_description,shot_quality_description,quality,game_goal_id,goal_type_name,goal_type,location_set,power_play,empty_net,penalty_shot,short_handed,insurance_goal,game_winning,game_tieing,scorer_goal_num,plus,minus,hit_type,player_served,offence,pp,time_off_formatted,bench,minutes,minutes_formatted,penalty_class_id,penalty_class,lang_penalty_description,game_id,orderIdx,event_detail,player1Id,player1JerseyNumber,player1FirstName,player1LastName,player1Team,player1TeamId,player2Id,player2JerseyNumber,player2FirstName,player2LastName,player2Team,player2TeamId,player3Id,player3JerseyNumber,player3FirstName,player3LastName,player3Team,player3TeamId,eventTeam,eventTeamId,goalieId,goalieJerseyNumber,goalieFirstName,goalieLastName,goalieTeam,goalieTeamId,n_plus,n_minus,homeSkaters,awaySkaters,plusPlayer1FirstName,plusPlayer2FirstName,plusPlayer3FirstName,plusPlayer4FirstName,plusPlayer5FirstName,plusPlayer1JerseyNumber,plusPlayer2JerseyNumber,plusPlayer3JerseyNumber,plusPlayer4JerseyNumber,plusPlayer5JerseyNumber,plusPlayer1LastName,plusPlayer2LastName,plusPlayer3LastName,plusPlayer4LastName,plusPlayer5LastName,plusPlayer1Id,plusPlayer2Id,plusPlayer3Id,plusPlayer4Id,plusPlayer5Id,plusPlayer1Team,plusPlayer2Team,plusPlayer3Team,plusPlayer4Team,plusPlayer5Team,plusPlayer1TeamId,plusPlayer2TeamId,plusPlayer3TeamId,plusPlayer4TeamId,plusPlayer5TeamId,minusPlayer1FirstName,minusPlayer2FirstName,minusPlayer3FirstName,minusPlayer4FirstName,minusPlayer5FirstName,minusPlayer1JerseyNumber,minusPlayer2JerseyNumber,minusPlayer3JerseyNumber,minusPlayer4JerseyNumber,minusPlayer5JerseyNumber,minusPlayer1LastName,minusPlayer2LastName,minusPlayer3LastName,minusPlayer4LastName,minusPlayer5LastName,minusPlayer1Id,minusPlayer2Id,minusPlayer3Id,minusPlayer4Id,minusPlayer5Id,minusPlayer1Team,minusPlayer2Team,minusPlayer3Team,minusPlayer4Team,minusPlayer5Team,minusPlayer1TeamId,minusPlayer2TeamId,minusPlayer3TeamId,minusPlayer4TeamId,minusPlayer5TeamId,elapsedTime,score_home,score_away,x_norm,y_norm,x,y,shot_distance_ft,shot_angle_deg,elapsedTime_prev,time_formatted_prev,event_prev,shot_type_prev,shot_type_description_prev,shot_quality_description_prev,quality_prev,game_goal_id_prev,penalty_shot_prev,location_set_prev,power_play_prev,empty_net_prev,short_handed_prev,insurance_goal_prev,game_winning_prev,game_tieing_prev,scorer_goal_num_prev,plus_prev,minus_prev,player2Id_prev,player2JerseyNumber_prev,player2FirstName_prev,player2LastName_prev,player2Team_prev,player2TeamId_prev,player3Id_prev,player3JerseyNumber_prev,player3FirstName_prev,player3LastName_prev,player3Team_prev,player3TeamId_prev,eventTeam_prev,eventTeamId_prev,goalieId_prev,goalieJerseyNumber_prev,goalieFirstName_prev,goalieLastName_prev,goalieTeam_prev,goalieTeamId_prev,plusPlayer1FirstName_prev,plusPlayer2FirstName_prev,plusPlayer3FirstName_prev,plusPlayer4FirstName_prev,plusPlayer5FirstName_prev,plusPlayer1JerseyNumber_prev,plusPlayer2JerseyNumber_prev,plusPlayer3JerseyNumber_prev,plusPlayer4JerseyNumber_prev,plusPlayer5JerseyNumber_prev,plusPlayer1LastName_prev,plusPlayer2LastName_prev,plusPlayer3LastName_prev,plusPlayer4LastName_prev,plusPlayer5LastName_prev,plusPlayer1Id_prev,plusPlayer2Id_prev,plusPlayer3Id_prev,plusPlayer4Id_prev,plusPlayer5Id_prev,plusPlayer1Team_prev,plusPlayer2Team_prev,plusPlayer3Team_prev,plusPlayer4Team_prev,plusPlayer5Team_prev,plusPlayer1TeamId_prev,plusPlayer2TeamId_prev,plusPlayer3TeamId_prev,plusPlayer4TeamId_prev,plusPlayer5TeamId_prev,minusPlayer1FirstName_prev,minusPlayer2FirstName_prev,minusPlayer3FirstName_prev,minusPlayer4FirstName_prev,minusPlayer5FirstName_prev,minusPlayer1JerseyNumber_prev,minusPlayer2JerseyNumber_prev,minusPlayer3JerseyNumber_prev,minusPlayer4JerseyNumber_prev,minusPlayer5JerseyNumber_prev,minusPlayer1LastName_prev,minusPlayer2LastName_prev,minusPlayer3LastName_prev,minusPlayer4LastName_prev,minusPlayer5LastName_prev,minusPlaye

In [5]:
# Demonstrate analysis-ready data
print("="*80)
print("ANALYSIS-READY DATAFRAME")
print("="*80)

# Show goals with complete shot information
goals = df[df['event'] == 'goal'][[
    'period', 'elapsedTime', 'player1FirstName', 'player1LastName', 
    'shot_type', 'x', 'y', 'goalieFirstName', 'goalieLastName',
    'score_home', 'score_away', 'power_play', 'empty_net'
]]

print(f"\nGoals in game 31171: {len(goals)}")
print("\nGoals with shot details:")
goals

ANALYSIS-READY DATAFRAME

Goals in game 31171: 6

Goals with shot details:


,period,elapsedTime,player1FirstName,player1LastName,shot_type,x,y,goalieFirstName,goalieLastName,score_home,score_away,power_play,empty_net
4,1.0,17.0,<NA>,<NA>,5,76.666667,7.666667,<NA>,<NA>,0,1,NaN,NaN
75,2.0,1474.0,<NA>,<NA>,5,79.000000,0.333333,<NA>,<NA>,1,1,NaN,NaN
81,2.0,1537.0,<NA>,<NA>,5,76.666667,20.666667,<NA>,<NA>,1,2,NaN,NaN
95,2.0,1688.0,<NA>,<NA>,5,78.000000,13.000000,<NA>,<NA>,2,2,NaN,NaN
112,2.0,1998.0,<NA>,<NA>,5,74.333333,-13.000000,<NA>,<NA>,3,2,NaN,NaN
139,3.0,2654.0,<NA>,<NA>,5,79.666667,3.000000,<NA>,<NA>,4,2,NaN,NaN


In [8]:
# Compare NHL-style vs QMJHL-style data
print("="*80)
print("NHLIFY COMPARISON")
print("="*80)

# NHL-style: one row per shot event (goals have shot data merged in)
df_nhl = scrape_game(31171, nhlify=True)
print(f"\nNHL-style (nhlify=True): {len(df_nhl)} rows")
print(f"Events: {dict(df_nhl['event'].value_counts())}")

# QMJHL-style: separate rows for shots and goals
df_qmjhl = scrape_game(31171, nhlify=False)
print(f"\nQMJHL-style (nhlify=False): {len(df_qmjhl)} rows")
print(f"Events: {dict(df_qmjhl['event'].value_counts())}")

print(f"\nDifference: {len(df_qmjhl) - len(df_nhl)} redundant shot rows removed with nhlify=True")
print("\nRecommendation: Use nhlify=True (default) for easier analysis and xG modeling")

NHLIFY COMPARISON

NHL-style (nhlify=True): 186 rows
Events: {'faceoff': np.int64(64), 'shot': np.int64(61), 'hit': np.int64(40), 'goalie_change': np.int64(8), 'penalty': np.int64(7), 'goal': np.int64(6)}

QMJHL-style (nhlify=False): 192 rows
Events: {'shot': np.int64(67), 'faceoff': np.int64(64), 'hit': np.int64(40), 'goalie_change': np.int64(8), 'penalty': np.int64(7), 'goal': np.int64(6)}

Difference: 6 redundant shot rows removed with nhlify=True

Recommendation: Use nhlify=True (default) for easier analysis and xG modeling


In [5]:
df.query("event in ['shot', 'goal']").head(2).filter(like=~'_prev')

TypeError: bad operand type for unary ~: 'str'

In [2]:
seasons_df = getSeasons()

seasons_df

,season_id,label,year,season_type
0,211,2025-26 | Regular Season,2026,regular
1,210,Pre-Season 2025,2026,preseason
2,209,2025 | Playoffs,2025,postseason
3,208,2024-25 | Regular Season,2025,regular
4,207,Pre-Season 2024,2025,preseason
...,...,...,...,...
123,87,2004 | Playoffs,2004,postseason
124,86,2003-04 | Regular Season,2004,regular
125,85,2003 | Playoffs,2003,postseason
126,84,2002-03 | Regular Season,2003,regular


In [3]:
season_id = 208

teams_df = getTeams(season_id=season_id)
teams_df

,season_id,team_id,team_label,city,nickname,url
0,208,2,"Acadie-Bathurst, Titan",Acadie-Bathurst,Titan,https://chl.ca/lhjmq/en/schedule/2/208/
1,208,16,"Baie-Comeau, Drakkar",Baie-Comeau,Drakkar,https://chl.ca/lhjmq/en/schedule/16/208/
2,208,19,"Blainville-Boisbriand, Armada",Blainville-Boisbriand,Armada,https://chl.ca/lhjmq/en/schedule/19/208/
3,208,3,"Cape Breton, Eagles",Cape Breton,Eagles,https://chl.ca/lhjmq/en/schedule/3/208/
4,208,7,"Charlottetown, Islanders",Charlottetown,Islanders,https://chl.ca/lhjmq/en/schedule/7/208/
5,208,10,"Chicoutimi, Saguenéens",Chicoutimi,Saguenéens,https://chl.ca/lhjmq/en/schedule/10/208/
6,208,14,"Drummondville, Voltigeurs",Drummondville,Voltigeurs,https://chl.ca/lhjmq/en/schedule/14/208/
7,208,12,"Gatineau, Olympiques",Gatineau,Olympiques,https://chl.ca/lhjmq/en/schedule/12/208/
8,208,5,"Halifax, Mooseheads",Halifax,Mooseheads,https://chl.ca/lhjmq/en/schedule/5/208/
9,208,1,"Moncton, Wildcats",Moncton,Wildcats,https://chl.ca/lhjmq/en/schedule/1/208/


In [4]:
teams_ids = teams_df['team_id'].tolist()

with ThreadPoolExecutor(max_workers=10) as executor:
    futures = {executor.submit(getSchedule, team_id=team_id, season_id=season_id): team_id for team_id in teams_ids}
    
    all_schedules = []
    for future in tqdm(futures, total=len(futures), desc="Scraping games"):
        team_id = futures[future]
        try:
            team_schedule = future.result()
            all_schedules.append(team_schedule)
        except Exception as e:
            print(f"Error scraping games for team {team_id}: {e}")
            
all_schedules_df = pd.concat(all_schedules, ignore_index=True)
all_schedules_df

Scraping games: 100%|██████████| 18/18 [00:01<00:00, 14.90it/s]


,ID,SeasonID,league_id,game_number,game_letter,game_type,quick_score,Date,FloCoreEventId,FloLiveEventId,GameDate,GameDateISO8601,ScheduledTime,ScheduledFormattedTime,Timezone,TicketUrl,HomeID,HomeCode,HomeCity,HomeNickname,HomeLongName,HomeDivision,HomeGoals,HomeAudioUrl,HomeVideoUrl,HomeWebcastUrl,VisitorID,VisitorCode,VisitorCity,VisitorNickname,VisitorLongName,VisitingDivision,VisitorGoals,VisitorAudioUrl,VisitorVideoUrl,VisitorWebcastUrl,Period,PeriodNameShort,PeriodNameLong,GameClock,GameSummaryUrl,HomeWins,HomeRegulationLosses,HomeOTLosses,HomeShootoutLosses,VisitorWins,VisitorRegulationLosses,VisitorOTLosses,VisitorShootoutLosses,GameStatus,Intermission,GameStatusString,GameStatusStringLong,Ord,venue_name,venue_location,league_name,league_code,TimezoneShort,HomeLogo,VisitorLogo,FloHockeyUrl,combined_client_code
0,31171,208,1,2,,,0,2024-09-20,,,"Fri, Sep 20",2024-09-20T19:00:00-03:00,19:00:00,7:00 pm,Canada/Atlantic,https://ticketboutik.evenue.net/cgi-bin/ncomme...,2,Bat,Acadie-Bathurst,Titan,"Acadie-Bathurst, Titan",Maritimes Division,4,https://ckle.fm/ecouter941,,https://watch.chl.ca/,3,Cap,Cape Breton,Eagles,"Cape Breton, Eagles",Maritimes Division,2,https://cjcbradio.com/,,https://watch.chl.ca/,3,3,3rd,00:00,31171,33,28,2,1,34,23,4,3,4,0,Final,Final,2024-09-20 18:00:00.000000,Centre régional K.C. Irving,Bathurst NB,Ligue de hockey junior Maritimes Québec,lhjmq,AT,https://assets.leaguestat.com/lhjmq/logos/2_20...,https://assets.leaguestat.com/lhjmq/logos/3.png,,None
1,31172,208,1,10,,,0,2024-09-21,,,"Sat, Sep 21",2024-09-21T16:00:00-03:00,16:00:00,4:00 pm,Canada/Atlantic,https://ticketboutik.evenue.net/cgi-bin/ncomme...,2,Bat,Acadie-Bathurst,Titan,"Acadie-Bathurst, Titan",Maritimes Division,4,https://ckle.fm/ecouter941,,https://watch.chl.ca/,3,Cap,Cape Breton,Eagles,"Cape Breton, Eagles",Maritimes Division,1,https://cjcbradio.com/,,https://watch.chl.ca/,3,3,3rd,00:00,31172,33,28,2,1,34,23,4,3,4,0,Final,Final,2024-09-21 15:00:00.000000,Centre régional K.C. Irving,Bathurst NB,Ligue de hockey junior Maritimes Québec,lhjmq,AT,https://assets.leaguestat.com/lhjmq/logos/2_20...,https://assets.leaguestat.com/lhjmq/logos/3.png,,None
2,31299,208,1,29,,,0,2024-09-28,,,"Sat, Sep 28",2024-09-28T19:00:00-03:00,19:00:00,7:00 pm,Canada/Atlantic,https://www.ticketmaster.ca/moncton-wildcats-t...,1,Mon,Moncton,Wildcats,"Moncton, Wildcats",Maritimes Division,2,https://inspire.fm/,,https://watch.chl.ca/,2,Bat,Acadie-Bathurst,Titan,"Acadie-Bathurst, Titan",Maritimes Division,3,https://ckle.fm/ecouter941,,https://watch.chl.ca/,4,OT1,1st OT,03:23,31299,53,9,2,0,33,28,2,1,4,0,Final,Final 1st OT,2024-09-28 18:00:00.000000,Avenir Centre,Moncton (NB),Ligue de hockey junior Maritimes Québec,lhjmq,AT,https://assets.leaguestat.com/lhjmq/logos/1.png,https://assets.leaguestat.com/lhjmq/logos/2_20...,,None
3,31173,208,1,30,,,0,2024-09-29,,,"Sun, Sep 29",2024-09-29T15:00:00-03:00,15:00:00,3:00 pm,Canada/Atlantic,https://ticketboutik.evenue.net/cgi-bin/ncomme...,2,Bat,Acadie-Bathurst,Titan,"Acadie-Bathurst, Titan",Maritimes Division,0,https://ckle.fm/ecouter941,,https://watch.chl.ca/,7,Cha,Charlottetown,Islanders,"Charlottetown, Islanders",Maritimes Division,2,https://max931.ca,,https://watch.chl.ca/,3,3,3rd,00:00,31173,33,28,2,1,30,29,4,1,4,0,Final,Final,2024-09-29 14:00:00.000000,Centre régional K.C. Irving,Bathurst NB,Ligue de hockey junior Maritimes Québec,lhjmq,AT,https://assets.leaguestat.com/lhjmq/logos/2_20...,https://assets.leaguestat.com/lhjmq/logos/7.png,,None
4,31174,208,1,38,,,0,2024-10-03,,,"Thu, Oct 3",2024-10-03T19:00:00-03:00,19:00:00,7:00 pm,Canada/Atlantic,https://ticketboutik.evenue.net/cgi-bin/ncomme...,2,Bat,Acadie-Bathurst,Titan,"Acadie-Bathurst, Titan",Maritimes Division,3,https://ckle.fm/ecouter941,,https://watch.chl.ca/,8,SNB,Saint John,Sea Dogs,"Saint John, Sea Dogs",Maritimes Division,6,https://chl.ca/lhjmq-seadogs/radio/,,https://watch.chl.ca/,3,3,3rd,00:00,31174,33,28,2,1,21,43,0,0,4,0,Final,Final,2024-10-03 18:00:00.000000

In [5]:
games_ids = all_schedules_df['ID'].unique()

games_ids[:10]

array(['31171', '31172', '31299', '31173', '31174', '31175', '31334',
       '31176', '31269', '31177'], dtype=object)

In [6]:
scrape_game(31632)

,id,goalie_id,event,home,time,time_formatted,x_location,y_location,shot_type,shot_type_description,shot_quality_description,quality,game_goal_id,s,period,hit_type,location_id,player_served,offence,pp,time_off_formatted,bench,penalty_shot,minutes,minutes_formatted,penalty_class_id,penalty_class,lang_penalty_description,shot_order,goal,winning_goal,goalie_info,shooter_info,game_id,orderIdx,event_detail,player1Id,player1JerseyNumber,player1FirstName,player1LastName,player1Team,player1TeamId,player2Id,player2JerseyNumber,player2FirstName,player2LastName,player2Team,player2TeamId,player3Id,player3JerseyNumber,player3FirstName,player3LastName,player3Team,player3TeamId,eventTeam,eventTeamId,goalieId,goalieJerseyNumber,goalieFirstName,goalieLastName,goalieTeam,goalieTeamId,plus,minus,n_plus,n_minus,homeSkaters,awaySkaters,elapsedTime,score_home,score_away,x_norm,y_norm,x,y,shot_distance_ft,shot_angle_deg,shot_type_prev,shot_type_description_prev,shot_quality_description_prev,quality_prev,game_goal_id_prev,penalty_shot_prev,plus_prev,minus_prev,player2Id_prev,player2JerseyNumber_prev,player2FirstName_prev,player2LastName_prev,player2Team_prev,player2TeamId_prev,player3Id_prev,player3JerseyNumber_prev,player3FirstName_prev,player3LastName_prev,player3Team_prev,player3TeamId_prev,eventTeam_prev,eventTeamId_prev,goalieId_prev,goalieJerseyNumber_prev,goalieFirstName_prev,goalieLastName_prev,goalieTeam_prev,goalieTeamId_prev,scrapeOn
0,639383,20198,shot,1.0,00:00,0:00,97.0,119.0,5,Default,Quality on net,1,,0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31632,0,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,NaN,NaN,<NA>,<NA>,<NA>,<NA>,NaN,NaN,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,NaN,NaN,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0.0,0,0,503.0,119.0,67.666667,-10.333333,33.944399,17.723237,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,2026-01-09 18:34:15.640426+00:00
1,639386,20198,shot,1.0,00:00,0:00,153.0,156.0,5,Default,Non quality on net,2,,0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31632,1,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,NaN,NaN,<NA>,<NA>,<NA>,<NA>,NaN,NaN,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,NaN,NaN,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0.0,0,0,447.0,156.0,49.000000,2.000000,51.039201,2.245743,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,2026-01-09 18:34:15.640426+00:00
2,322895,NaN,hit,1.0,00:00:00,0:00,205.0,18.0,NaN,NaN,NaN,NaN,NaN,0,1.0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31632,2,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,NaN,NaN,<NA>,<NA>,<NA>,<NA>,NaN,NaN,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,NaN,NaN,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0.0,0,0,395.0,18.0,31.666667,-44.000000,81.273885,32.777584,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,2026-01-09 18:34:15.640426+00:00
3,NaN,NaN,goalie_change,NaN,0:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31632,3,in,NaN,<NA>,<NA>,<NA>,<NA>,NaN,NaN,<NA>,<NA>,<NA>,<NA>,NaN,NaN,<NA>,<NA>,<NA>,<NA>,NaN,BaC,16.0,NaN,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,2026-01-09 18:34:15.640426+00:00
4,NaN,NaN,goalie_change,NaN,0:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31632,4,in,NaN,<NA>,<NA>,<NA>,<NA>,NaN,NaN,<NA>,<NA>,<NA>,<NA>,NaN,NaN,<NA>,<NA>,<NA>,<NA>,NaN,Sha,13.0,NaN,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA

In [ ]:
# Test batch scraping with improved clean_pbp
games_ids = all_schedules_df['ID'].unique()[:10]  # Test with first 10 games

with ThreadPoolExecutor(max_workers=10) as executor:
    futures = {executor.submit(scrape_game, game_id=game_id): game_id for game_id in games_ids}
    
    all_games = []
    for future in tqdm(futures, total=len(futures), desc="Scraping games"):
        game_id = futures[future]
        try:
            game_data = future.result()
            all_games.append(game_data)
        except Exception as e:
            print(f"Error scraping game {game_id}: {e}")    

all_games_df = pd.concat(all_games, ignore_index=True)
print(f"\nScraped {len(all_games)} games with {len(all_games_df)} total events")
print(f"\nEvent distribution:\n{all_games_df['event'].value_counts()}")
all_games_df.head()

Scraping games: 100%|██████████| 576/576 [00:53<00:00, 10.84it/s]
/var/folders/8b/dctd609n4zsgx5052t61l01c0000gn/T/ipykernel_66913/4190375312.py:14: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_games_df = pd.concat(all_games, ignore_index=True)


,event,time,s,id,period,time_formatted,location_id,x_location,y_location,goalie_id,home,shot_type,shot_type_description,shot_quality_description,quality,game_goal_id,goal_type_name,goal_type,location_set,power_play,empty_net,penalty_shot,short_handed,insurance_goal,game_winning,game_tieing,scorer_goal_num,plus,minus,hit_type,player_served,offence,pp,time_off_formatted,bench,minutes,minutes_formatted,penalty_class_id,penalty_class,lang_penalty_description,game_id,orderIdx,event_detail,player1Id,player1JerseyNumber,player1FirstName,player1LastName,player1Team,player1TeamId,player2Id,player2JerseyNumber,player2FirstName,player2LastName,player2Team,player2TeamId,player3Id,player3JerseyNumber,player3FirstName,player3LastName,player3Team,player3TeamId,eventTeam,eventTeamId,goalieId,goalieJerseyNumber,goalieFirstName,goalieLastName,goalieTeam,goalieTeamId,n_plus,n_minus,homeSkaters,awaySkaters,plusPlayer1FirstName,plusPlayer2FirstName,plusPlayer3FirstName,plusPlayer4FirstName,plusPlayer5FirstName,plusPlayer1JerseyNumber,plusPlayer2JerseyNumber,plusPlayer3JerseyNumber,plusPlayer4JerseyNumber,plusPlayer5JerseyNumber,plusPlayer1LastName,plusPlayer2LastName,plusPlayer3LastName,plusPlayer4LastName,plusPlayer5LastName,plusPlayer1Id,plusPlayer2Id,plusPlayer3Id,plusPlayer4Id,plusPlayer5Id,plusPlayer1Team,plusPlayer2Team,plusPlayer3Team,plusPlayer4Team,plusPlayer5Team,plusPlayer1TeamId,plusPlayer2TeamId,plusPlayer3TeamId,plusPlayer4TeamId,plusPlayer5TeamId,minusPlayer1FirstName,minusPlayer2FirstName,minusPlayer3FirstName,minusPlayer4FirstName,minusPlayer5FirstName,minusPlayer1JerseyNumber,minusPlayer2JerseyNumber,minusPlayer3JerseyNumber,minusPlayer4JerseyNumber,minusPlayer5JerseyNumber,minusPlayer1LastName,minusPlayer2LastName,minusPlayer3LastName,minusPlayer4LastName,minusPlayer5LastName,minusPlayer1Id,minusPlayer2Id,minusPlayer3Id,minusPlayer4Id,minusPlayer5Id,minusPlayer1Team,minusPlayer2Team,minusPlayer3Team,minusPlayer4Team,minusPlayer5Team,minusPlayer1TeamId,minusPlayer2TeamId,minusPlayer3TeamId,minusPlayer4TeamId,minusPlayer5TeamId,elapsedTime,score_home,score_away,x_norm,y_norm,x,y,shot_distance_ft,shot_angle_deg,shot_type_prev,shot_type_description_prev,shot_quality_description_prev,quality_prev,game_goal_id_prev,penalty_shot_prev,location_set_prev,power_play_prev,empty_net_prev,short_handed_prev,insurance_goal_prev,game_winning_prev,game_tieing_prev,scorer_goal_num_prev,plus_prev,minus_prev,player2Id_prev,player2JerseyNumber_prev,player2FirstName_prev,player2LastName_prev,player2Team_prev,player2TeamId_prev,player3Id_prev,player3JerseyNumber_prev,player3FirstName_prev,player3LastName_prev,player3Team_prev,player3TeamId_prev,eventTeam_prev,eventTeamId_prev,goalieId_prev,goalieJerseyNumber_prev,goalieFirstName_prev,goalieLastName_prev,goalieTeam_prev,goalieTeamId_prev,plusPlayer1FirstName_prev,plusPlayer2FirstName_prev,plusPlayer3FirstName_prev,plusPlayer4FirstName_prev,plusPlayer5FirstName_prev,plusPlayer1JerseyNumber_prev,plusPlayer2JerseyNumber_prev,plusPlayer3JerseyNumber_prev,plusPlayer4JerseyNumber_prev,plusPlayer5JerseyNumber_prev,plusPlayer1LastName_prev,plusPlayer2LastName_prev,plusPlayer3LastName_prev,plusPlayer4LastName_prev,plusPlayer5LastName_prev,plusPlayer1Id_prev,plusPlayer2Id_prev,plusPlayer3Id_prev,plusPlayer4Id_prev,plusPlayer5Id_prev,plusPlayer1Team_prev,plusPlayer2Team_prev,plusPlayer3Team_prev,plusPlayer4Team_prev,plusPlayer5Team_prev,plusPlayer1TeamId_prev,plusPlayer2TeamId_prev,plusPlayer3TeamId_prev,plusPlayer4TeamId_prev,plusPlayer5TeamId_prev,minusPlayer1FirstName_prev,minusPlayer2FirstName_prev,minusPlayer3FirstName_prev,minusPlayer4FirstName_prev,minusPlayer5FirstName_prev,minusPlayer1JerseyNumber_prev,minusPlayer2JerseyNumber_prev,minusPlayer3JerseyNumber_prev,minusPlayer4JerseyNumber_prev,minusPlayer5JerseyNumber_prev,minusPlayer1LastName_prev,minusPlayer2LastName_prev,minusPlayer3LastName_prev,minusPlayer4LastName_prev,minusPlayer5LastName_prev,minusPlayer1Id_prev,minusPlayer2Id_prev,minusPlayer3Id_pre

In [ ]:
# Analysis-ready: Get all shots and goals
shots_goals = all_games_df[all_games_df['event'].isin(['shot', 'goal', 'penaltyshot'])]

print(f"Total shooting events: {len(shots_goals)}")
print(f"  - Goals: {len(shots_goals[shots_goals['event'] == 'goal'])}")
print(f"  - Shots: {len(shots_goals[shots_goals['event'] == 'shot'])}")
print(f"  - Penalty shots: {len(shots_goals[shots_goals['event'] == 'penaltyshot'])}")
print(f"\nShooting %: {len(shots_goals[shots_goals['event'] == 'goal']) / len(shots_goals) * 100:.2f}%")

# Check data completeness for xG modeling
print(f"\nData completeness for xG modeling:")
print(f"  - Shot type available: {shots_goals['shot_type'].notna().sum()} ({shots_goals['shot_type'].notna().sum() / len(shots_goals) * 100:.1f}%)")
print(f"  - Coordinates available: {shots_goals['x'].notna().sum()} ({shots_goals['x'].notna().sum() / len(shots_goals) * 100:.1f}%)")
print(f"  - Distance available: {shots_goals['shot_distance_ft'].notna().sum()} ({shots_goals['shot_distance_ft'].notna().sum() / len(shots_goals) * 100:.1f}%)")
print(f"  - Angle available: {shots_goals['shot_angle_deg'].notna().sum()} ({shots_goals['shot_angle_deg'].notna().sum() / len(shots_goals) * 100:.1f}%)")

shots_goals.head()

,event,time,s,id,period,time_formatted,location_id,x_location,y_location,goalie_id,home,shot_type,shot_type_description,shot_quality_description,quality,game_goal_id,goal_type_name,goal_type,location_set,power_play,empty_net,penalty_shot,short_handed,insurance_goal,game_winning,game_tieing,scorer_goal_num,plus,minus,hit_type,player_served,offence,pp,time_off_formatted,bench,minutes,minutes_formatted,penalty_class_id,penalty_class,lang_penalty_description,game_id,orderIdx,event_detail,player1Id,player1JerseyNumber,player1FirstName,player1LastName,player1Team,player1TeamId,player2Id,player2JerseyNumber,player2FirstName,player2LastName,player2Team,player2TeamId,player3Id,player3JerseyNumber,player3FirstName,player3LastName,player3Team,player3TeamId,eventTeam,eventTeamId,goalieId,goalieJerseyNumber,goalieFirstName,goalieLastName,goalieTeam,goalieTeamId,n_plus,n_minus,homeSkaters,awaySkaters,plusPlayer1FirstName,plusPlayer2FirstName,plusPlayer3FirstName,plusPlayer4FirstName,plusPlayer5FirstName,plusPlayer1JerseyNumber,plusPlayer2JerseyNumber,plusPlayer3JerseyNumber,plusPlayer4JerseyNumber,plusPlayer5JerseyNumber,plusPlayer1LastName,plusPlayer2LastName,plusPlayer3LastName,plusPlayer4LastName,plusPlayer5LastName,plusPlayer1Id,plusPlayer2Id,plusPlayer3Id,plusPlayer4Id,plusPlayer5Id,plusPlayer1Team,plusPlayer2Team,plusPlayer3Team,plusPlayer4Team,plusPlayer5Team,plusPlayer1TeamId,plusPlayer2TeamId,plusPlayer3TeamId,plusPlayer4TeamId,plusPlayer5TeamId,minusPlayer1FirstName,minusPlayer2FirstName,minusPlayer3FirstName,minusPlayer4FirstName,minusPlayer5FirstName,minusPlayer1JerseyNumber,minusPlayer2JerseyNumber,minusPlayer3JerseyNumber,minusPlayer4JerseyNumber,minusPlayer5JerseyNumber,minusPlayer1LastName,minusPlayer2LastName,minusPlayer3LastName,minusPlayer4LastName,minusPlayer5LastName,minusPlayer1Id,minusPlayer2Id,minusPlayer3Id,minusPlayer4Id,minusPlayer5Id,minusPlayer1Team,minusPlayer2Team,minusPlayer3Team,minusPlayer4Team,minusPlayer5Team,minusPlayer1TeamId,minusPlayer2TeamId,minusPlayer3TeamId,minusPlayer4TeamId,minusPlayer5TeamId,elapsedTime,score_home,score_away,x_norm,y_norm,x,y,shot_distance_ft,shot_angle_deg,shot_type_prev,shot_type_description_prev,shot_quality_description_prev,quality_prev,game_goal_id_prev,penalty_shot_prev,location_set_prev,power_play_prev,empty_net_prev,short_handed_prev,insurance_goal_prev,game_winning_prev,game_tieing_prev,scorer_goal_num_prev,plus_prev,minus_prev,player2Id_prev,player2JerseyNumber_prev,player2FirstName_prev,player2LastName_prev,player2Team_prev,player2TeamId_prev,player3Id_prev,player3JerseyNumber_prev,player3FirstName_prev,player3LastName_prev,player3Team_prev,player3TeamId_prev,eventTeam_prev,eventTeamId_prev,goalieId_prev,goalieJerseyNumber_prev,goalieFirstName_prev,goalieLastName_prev,goalieTeam_prev,goalieTeamId_prev,plusPlayer1FirstName_prev,plusPlayer2FirstName_prev,plusPlayer3FirstName_prev,plusPlayer4FirstName_prev,plusPlayer5FirstName_prev,plusPlayer1JerseyNumber_prev,plusPlayer2JerseyNumber_prev,plusPlayer3JerseyNumber_prev,plusPlayer4JerseyNumber_prev,plusPlayer5JerseyNumber_prev,plusPlayer1LastName_prev,plusPlayer2LastName_prev,plusPlayer3LastName_prev,plusPlayer4LastName_prev,plusPlayer5LastName_prev,plusPlayer1Id_prev,plusPlayer2Id_prev,plusPlayer3Id_prev,plusPlayer4Id_prev,plusPlayer5Id_prev,plusPlayer1Team_prev,plusPlayer2Team_prev,plusPlayer3Team_prev,plusPlayer4Team_prev,plusPlayer5Team_prev,plusPlayer1TeamId_prev,plusPlayer2TeamId_prev,plusPlayer3TeamId_prev,plusPlayer4TeamId_prev,plusPlayer5TeamId_prev,minusPlayer1FirstName_prev,minusPlayer2FirstName_prev,minusPlayer3FirstName_prev,minusPlayer4FirstName_prev,minusPlayer5FirstName_prev,minusPlayer1JerseyNumber_prev,minusPlayer2JerseyNumber_prev,minusPlayer3JerseyNumber_prev,minusPlayer4JerseyNumber_prev,minusPlayer5JerseyNumber_prev,minusPlayer1LastName_prev,minusPlayer2LastName_prev,minusPlayer3LastName_prev,minusPlayer4LastName_prev,minusPlayer5LastName_prev,minusPlayer1Id_prev,minusPlayer2Id_prev,minusPlayer3Id_pre